# Reddit

Para utilização do scrapper é preciso cadastrar um cliente na API do reddit, para obter as credênciais de acesso.

É possível obter esses tokens ao cadastrar a aplicação nessa página: https://www.reddit.com/prefs/apps/

In [1]:
import praw
import pandas as pd
from datetime import datetime

# Tokens de Acesso a API do Reddit
# Criada para Uso do Exclusivo
reddit = praw.Reddit(client_id='Zahz6NxvpyRV4A', client_secret='SvvYqBQecVzEJwTwvq3kGy6l-zg', user_agent='TCC - Reddit Scrapper')

# Link de um post do reddit
# Também pode ser substituido por tópicos da página principal, ou de algum subreddit.
submission = reddit.submission(url="https://www.reddit.com/r/news/comments/i0lu1y/donald_trump_calls_for_delay_to_2020_us/")


In [2]:
from praw.models import MoreComments

def get_first_level_comments(list_of_comments, index,dataframe):
    for comentario in list_of_comments:
        if(type(comentario) == praw.models.reddit.more.MoreComments):
            index = get_first_level_comments(comentario.comments(), index, dataframe)
            continue
        
        print_root_info(comentario, index, dataframe)
        
        if(comentario.replies):
            child_id = index + 1
            
            last_id = get_replies_from_node(comentario.replies, index, child_id, dataframe)
            
            index = last_id + 1
        else:
            index += 1
    
    # Retorna o indice do último comentário +1
    return index


In [3]:
def get_replies_from_node(replies, in_reply_to_id, node_id, dataframe):
    index = node_id
    for resposta in replies:
        if(type(resposta) == praw.models.reddit.more.MoreComments):
            return get_replies_from_node(resposta.comments(), in_reply_to_id, index, dataframe)
        
        print_node_info(resposta, index, in_reply_to_id, dataframe)
        
        if(resposta.replies):
            child_id = index + 1
            last_id = get_replies_from_node(resposta.replies, index, child_id, dataframe)
              
            index = last_id + 1
            
        else:
            index += 1
    
    return index - 1
            

def print_root_info(comentario, index, dataframe):
    date = datetime.fromtimestamp(comentario.created_utc)
    body = str(comentario.body).replace("\n", " ")
    body = body.replace("\t"," ")

#     print(index, comentario.author, date.strftime('%d/%m/%Y'), date.strftime('%H:%M:%S'),comentario.body, "postion", "''")
    dataframe.loc[index] = [index, str(comentario.author),date.strftime('%d/%m/%Y'), date.strftime('%H:%M:%S'), body, submission.title, 'neutral', '']
    

def print_node_info(resposta, index, in_reply_to_id, dataframe):
    date = datetime.fromtimestamp(resposta.created_utc)
    body = str(resposta.body).replace("\n", "")
    body = body.replace("\t"," ")

#     print(f"{index} {resposta.author} {date.strftime('%d/%m/%Y')} {date.strftime('%H:%M:%S')} {resposta.body} {'position'} {in_reply_to_id}")
    dataframe.loc[index] = [index, str(resposta.author),date.strftime('%d/%m/%Y'), date.strftime('%H:%M:%S'), body, submission.title, 'neutral', in_reply_to_id]

In [4]:
df = pd.DataFrame(columns=['id', 'sender', 'date', 'time', 'content', 'topic', 'position', 'addressee'])

get_first_level_comments(submission.comments,1, df)

df.to_csv("reddit_post_extraction.csv", index_label=False)



# Twitter

Teste de outro método de extração, utilizando o Twitter como rede social

In [ ]:
import tweepy

ACCESS_TOKEN = ''
ACCESS_SECRET = ''
CONSUMER_KEY = ''
CONSUMER_SECRET = ''


# Setup access to API
def connect_to_twitter_OAuth():
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    api = tweepy.API(auth)
    return api

api = connect_to_twitter_OAuth()

trump_tweets = api.user_timeline('rakin')
for tweet in trump_tweets:
    print(tweet.text, tweet.id)

api.me()

#tweets = api.search()

### O código funciona para pegar as replies, porém se o tweet for relativamente antigo, ou a pessoa tem muita atividade na plataforma, há uma demora excessiva para retornar as respostas

### Isso acontece por conta da maneira que o método da api está implementado


In [ ]:
def get_replies(user, status_id):
    replies = tweepy.Cursor(api.search, q='to:{}'.format(user),
                                since_id=status_id, tweet_mode='extended').items()
    resp = []
    while True:
        try:
            reply = replies.next()
            if not hasattr(reply, 'in_reply_to_status_id_str'):
                continue
            if reply.in_reply_to_status_id == status_id:
               # logging.info("reply of tweet:{}".format(reply.full_text))
                aux = "@" + str(reply.user.screen_name)
                aux_id = reply.id
                print(reply.full_text)
                get_replies(aux, aux_id)
                resp.append({"user": "@" + str(reply.user.screen_name), "id": reply.id, "text": reply.full_text, "tweet_replies": []})
                
        except tweepy.RateLimitError as e:
            print("Twitter api rate limit reached".format(e))
            # time.sleep(60)
            continue

        except tweepy.TweepError as e:
            print("Tweepy error occured:{}".format(e))
            break

        except StopIteration:
            break

        except Exception as e:
            print("Failed while fetching replies {}".format(e))
            break
    
    return resp

In [ ]:
user_name="@HaruJiggly"
tweet_id = 1282782489719779337

get_replies(user_name, tweet_id)
retorno = get_replies(user_name, tweet_id)
for i in retorno:
    print(i)
    

# def get_sub_replies(reply_list_of_dicts):
#     for item in reply_list_of_dicts:
#         print(item["user"], item["id"], get_replies(item["user"], item["id"]))

# get_sub_replies(retorno)
    